### Notebook to demonstrate Image Classification workflow

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)

### Sample prediction for an Image Classification model
<img align="center" src="../example_images/sample_image_classification.jpg">

### The workflow in a nutshell

- Creating a dataset
- Upload dataset to the service
- Getting a PTM from NGC
- Model Actions
    - Train (Normal/AutoML)
    - Evaluate
    - Prune, retrain
    - Export
    - TAO-Deploy
    - Inference on TAO

### Table of contents

1. [Create datasets ](#head-1)
1. [List the created datasets](#head-2)
1. [Create model ](#head-4)
1. [List models](#head-5)
1. [Assign train, eval datasets](#head-6)
1. [Assign PTM](#head-7)
1. [View hyperparameters that are enabled by default](#head-8)
1. [Set AutoML related configurations](#head-9)
1. [Actions](#head-10)
1. [Train](#head-11)
1. [Evaluate](#head-12)
1. [Optimize: Apply specs for prune](#head-14)
1. [Optimize: Apply specs for retrain](#head-15)
1. [Optimize: Run actions](#head-16)
1. [Export](#head-17)
1. [TRT Engine generation using TAO-Deploy](#head-19)
1. [TAO inference](#head-20)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import json
import os
import requests
import uuid
import time
from IPython.display import clear_output
import subprocess
import glob

### FIXME

1. Assign a model_name in FIXME 1
1. Assign a workdir in FIXME 2
1. Assign the ip_address and port_number in FIXME 3 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
1. Assign the ngc_api_key variable in FIXME 4
1. (Optional) Enable AutoML if needed in FIXME 5
1. (Optional) Choose between Bayesian and Hyperband automl_algorithm in FIXME 6 (If automl was enabled in FIXME5)
1. Choose to download jobs or not in FIXME 7
1. Choose between default and custom dataset in FIXME 8
1. Assign path of DATA_DIR in FIXME 9

In [ ]:
# Define model_name workspaces and other variables
# Available models (#FIXME 1):
# 1. classification_pyt - https://docs.nvidia.com/tao/tao-toolkit/text/image_classification.html
# 2. classification_tf1 - https://docs.nvidia.com/tao/tao-toolkit/text/image_classification.html
# 3. classification_tf2 - https://docs.nvidia.com/tao/tao-toolkit/text/image_classification_tf2.html
# 4. multitask_classification - https://docs.nvidia.com/tao/tao-toolkit/text/multitask_image_classification.html
# classification is the same as multi-class classification

model_name = "multitask_classification" # FIXME1 (Add the model name from the above mentioned list)

In [ ]:
workdir = "workdir_classification" # FIXME2
host_url = "http://<ip_address>:<port_number>" # FIXME3 example: https://10.137.149.22:32334
# In host machine, node ip_address and port number can be obtained as follows,
# ip_address: hostname -i
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'
ngc_api_key = "<ngc_api_key>" # FIXME4 example: (Add NGC API key)

In [ ]:
automl_enabled = False # FIXME5 set to True if you want to run automl for the model chosen in the previous cell
automl_algorithm = "Bayesian" # FIXME6 example: Bayesian/HyperBand
# FIXME7 Defaulted to False as downloading jobs from service to your machine takes time
# Set to True if you want to download jobs where examples have been provided like for train, export, inference.
download_jobs = False

In [ ]:
# Exchange NGC_API_KEY for JWT
data = json.dumps({"ngc_api_key": ngc_api_key})
response = requests.post(f"{host_url}/api/v1/login", data=data)
assert response.status_code in (200, 201)
assert "user_id" in response.json().keys()
user_id = response.json()["user_id"]
print("User ID",user_id)
assert "token" in response.json().keys()
token = response.json()["token"]
print("JWT",token)

# Set base URL
base_url = f"{host_url}/api/v1/user/{user_id}"
print("API Calls will be forwarded to",base_url)

headers = {"Authorization": f"Bearer {token}"}

In [ ]:
# Creating workdir
if not os.path.isdir(workdir):
    os.makedirs(workdir)

### Function to split tar files <a class="anchor" id="head-1.1"></a>

In [ ]:
import os
import tarfile

def split_tar_file(input_tar_path, output_dir, max_split_size=1.5*1024*1024*1024):
	os.makedirs(output_dir, exist_ok=True)
	
	with tarfile.open(input_tar_path, 'r') as original_tar:
		members = original_tar.getmembers()
		current_split_size = 0
		current_split_number = 0
		current_split_name = os.path.join(output_dir, f'smaller_file_{current_split_number}.tar')
		
		with tarfile.open(current_split_name, 'w') as split_tar:
			for member in members:
				if current_split_size + member.size <= max_split_size:
					split_tar.addfile(member, original_tar.extractfile(member))
					current_split_size += member.size
				else:
					split_tar.close()
					current_split_number += 1
					current_split_name = os.path.join(output_dir, f'smaller_file_{current_split_number}.tar')
					current_split_size = 0
					split_tar = tarfile.open(current_split_name, 'w')  # Open a new split tar archive
					split_tar.addfile(member, original_tar.extractfile(member))
					current_split_size += member.size

### Set dataset type, format <a class="anchor" id="head-1.1"></a>

**For multi-class classification:**

We will be using the pascal `VOC dataset` for the tutorial. To find more details please visit [here](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/index.html#devkit). Please download the [dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar) to the environment variable `$DATA_DIR`.

**If using custom dataset; it should follow this dataset structure, and skip running** "**Split dataset into train and val sets**"
```
DATA_DIR
├── classes.txt
├── images_test
│   ├── class_name_1
│   │   ├── image_name_1.jpg
│   │   ├── image_name_2.jpg
│   │   ├── ...
|   |   ... 
│   └── class_name_n
│       ├── image_name_3.jpg
│       ├── image_name_4.jpg
│       ├── ...
├── images_train
│   ├── class_name_1
│   │   ├── image_name_5.jpg
│   │   ├── image_name_6.jpg
|   |   ...
│   └── class_name_n
│       ├── image_name_7.jpg
│       ├── image_name_8.jpg
│       ├── ...
|
└── images_val
    ├── class_name_1
    │   ├── image_name_9.jpg
    │   ├── image_name_10.jpg
    │   ├── ...
    |   ...
    └── class_name_n
        ├── image_name_11.jpg
        ├── image_name_12.jpg
        ├── ...
```
- Each class name folder should contain the images corresponding to that class
- Same class name folders should be present across images_test, images_train and images_val
- classes.txt is a file which contains the names of all classes (each name in a separate line)

**For multi-task classification:**

We will be using the `Fashion Product Images (Small)` for the tutorial. This dataset is available on Kaggle.In this tutorial, our trained classification network will perform three tasks: article category classification, base color classification and target season classification.

To download the dataset, you will need a Kaggle account. After login, you can download the dataset zip file [here](https://www.kaggle.com/paramaggarwal/fashion-product-images-small). The downloaded file is archive.zip with a subfolder called myntradataset. Unzip contents in this subfolder to your workdir created in the cell above and you should have a folder called images and a CSV file called styles.csv

**If using custom dataset; it should follow this dataset structure**
```
DATA_DIR
├── images
│   ├── image_name_1.jpg
│   ├── image_name_2.jpg
|   |   ├── ...
├── styles.csv
```

In [ ]:
# Create train dataset
ds_type = "image_classification"
if model_name == "classification_pyt":
    ds_format = model_name
elif "classification_" in model_name:
    ds_format = "default"
elif model_name == "multitask_classification":
    ds_format = "custom"

In [ ]:
dataset_to_be_used = "default" #FIXME8 #default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset
DATA_DIR = model_name # FIXME9
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p $DATA_DIR
job_map = {}

### Dataset download and pre-processing <a class="anchor" id="head-1"></a>

In [ ]:
if dataset_to_be_used == "default":
    if "classification_" in model_name:
        assert os.path.exists(os.path.join(DATA_DIR,"VOCtrainval_11-May-2012.tar"))
        !tar -xf $DATA_DIR/VOCtrainval_11-May-2012.tar -C $DATA_DIR
        assert (os.path.exists(f"{DATA_DIR}/VOCdevkit/"))
        !rm -rf $DATA_DIR/split
    elif model_name == "multitask_classification":
        assert os.path.exists(os.path.join(DATA_DIR,"archive.zip"))
        !unzip -uq $DATA_DIR/archive.zip -d $DATA_DIR/
        assert (os.path.exists(f"{DATA_DIR}/images"))
        assert (os.path.exists(f"{DATA_DIR}/styles.csv"))
        # Create subdirectories and remove existing files in them
        !mkdir -p $DATA_DIR/images_train && rm -rf $DATA_DIR/images_train/*
        !mkdir -p $DATA_DIR/images_val && rm -rf $DATA_DIR/images_val/*
        !mkdir -p $DATA_DIR/images_test && rm -rf $DATA_DIR/images_test/*

#### Split dataset into train and val sets

In [ ]:
# Split dataset into train and val sets
!python3 -m pip install numpy pandas==1.5.1 tqdm
if "classification_" in model_name and dataset_to_be_used == "default":
    !python3 ../dataset_prepare/classification/dataset_split.py
    assert (os.path.exists(f"{DATA_DIR}/split/images_train/"))
    assert (os.path.exists(f"{DATA_DIR}/split/images_val/"))
    assert (os.path.exists(f"{DATA_DIR}/split/images_test/"))
elif model_name == "multitask_classification" and dataset_to_be_used == "default":
    !python3 ../dataset_prepare/multitask_classification/dataset_split.py
    assert (os.path.exists(f"{DATA_DIR}/images_train/"))
    assert (os.path.exists(f"{DATA_DIR}/images_val/"))
    assert (os.path.exists(f"{DATA_DIR}/images_test/"))
    assert (os.path.exists(f"{DATA_DIR}/train.csv"))
    assert (os.path.exists(f"{DATA_DIR}/val.csv"))

### Create Tar files to upload

In [ ]:
if "classification_" in model_name:
    !tar -C $DATA_DIR/split/ -czf classification_train.tar.gz images_train classes.txt
    !tar -C $DATA_DIR/split/ -czf classification_val.tar.gz images_val classes.txt
    !tar -C $DATA_DIR/split/ -czf classification_test.tar.gz images_test classes.txt
elif model_name == "multitask_classification":
    !tar -C $DATA_DIR/ -czf mt_classification_train.tar.gz images_train train.csv val.csv
    !tar -C $DATA_DIR/ -czf mt_classification_val.tar.gz images_val val.csv
    !tar -C $DATA_DIR/ -czf mt_classification_test.tar.gz images_test

In [ ]:
if "classification_" in model_name:
    train_dataset_path =  "classification_train.tar.gz"
    eval_dataset_path = "classification_val.tar.gz"
    test_dataset_path = "classification_test.tar.gz"
elif model_name == "multitask_classification":
    train_dataset_path =  "mt_classification_train.tar.gz"
    eval_dataset_path = "mt_classification_val.tar.gz"
    test_dataset_path = "mt_classification_test.tar.gz"

### Create and upload train dataset <a class="anchor" id="head-1.2"></a>

In [ ]:
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)
assert response.status_code in (200, 201)


print(response)
print(response.json())
assert "id" in response.json().keys()
train_dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Train dataset",
                       "description":"My train dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{train_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)
assert response.status_code in (200, 201)


print(response)
print(response.json())

In [ ]:
# Upload
output_dir = os.path.join(os.path.dirname(os.path.abspath(train_dataset_path)), model_name, "train")
split_tar_file(train_dataset_path, output_dir)
for idx, tar_dataset_path in enumerate(os.listdir(output_dir)):
    print(f"Uploading {idx+1}/{len(os.listdir(output_dir))} tar split")
    files = [("file",open(os.path.join(output_dir, tar_dataset_path),"rb"))]

    endpoint = f"{base_url}/dataset/{train_dataset_id}/upload"

    response = requests.post(endpoint, files=files, headers=headers)
    assert response.status_code in (200, 201)
    assert "message" in response.json().keys() and response.json()["message"] == "Data successfully uploaded"

    print(response)
    print(response.json())

### Create and upload val dataset <a class="anchor" id="head-1.3"></a>

In [ ]:
# Create eval dataset
ds_type = "image_classification"
if model_name == "classification_pyt":
    ds_format = model_name
else:
    ds_format = "default"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())
assert "id" in response.json().keys()
eval_dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Eval dataset",
                       "description":"My eval dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{eval_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())

In [ ]:
# Upload
output_dir = os.path.join(os.path.dirname(os.path.abspath(eval_dataset_path)), model_name, "eval")
split_tar_file(eval_dataset_path, output_dir)
for idx, tar_dataset_path in enumerate(os.listdir(output_dir)):
    print(f"Uploading {idx+1}/{len(os.listdir(output_dir))} tar split")
    files = [("file",open(os.path.join(output_dir, tar_dataset_path),"rb"))]

    endpoint = f"{base_url}/dataset/{eval_dataset_id}/upload"

    response = requests.post(endpoint, files=files, headers=headers)
    assert response.status_code in (200, 201)
    assert "message" in response.json().keys() and response.json()["message"] == "Data successfully uploaded"

    print(response)
    print(response.json())

### Create and upload test dataset <a class="anchor" id="head-1.4"></a>

In [ ]:
 # Create testing dataset for inference
ds_type = "image_classification"
ds_format = "raw"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())
assert "id" in response.json().keys()
test_dataset_id = response.json()["id"]

In [ ]:
# Upload
output_dir = os.path.join(os.path.dirname(os.path.abspath(test_dataset_path)), model_name, "test")
split_tar_file(test_dataset_path, output_dir)
for idx, tar_dataset_path in enumerate(os.listdir(output_dir)):
    print(f"Uploading {idx+1}/{len(os.listdir(output_dir))} tar split")
    files = [("file",open(os.path.join(output_dir, tar_dataset_path),"rb"))]

    endpoint = f"{base_url}/dataset/{test_dataset_id}/upload"

    response = requests.post(endpoint, files=files, headers=headers)
    assert response.status_code in (200, 201)
    assert "message" in response.json().keys() and response.json()["message"] == "Data successfully uploaded"

    print(response)
    print(response.json())

### List the created datasets <a class="anchor" id="head-2"></a>

In [ ]:
endpoint = f"{base_url}/dataset"

response = requests.get(endpoint, headers=headers)
assert response.status_code in (200, 201)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("id\t\t\t\t\t type\t\t\t format\t\t name")
for rsp in response.json():
    rsp_keys = rsp.keys()
    assert "id" in rsp_keys
    assert "type" in rsp_keys
    assert "format" in rsp_keys
    assert "name" in rsp_keys
    print(rsp["id"],"\t",rsp["type"],"\t",rsp["format"],"\t\t",rsp["name"])

### Create model <a class="anchor" id="head-4"></a>

In [ ]:
if "classification" in model_name:
    encode_key = "nvidia_tlt"
else:
    encode_key = "tlt_encode"
checkpoint_choose_method = "best_model"
data = json.dumps({"network_arch":model_name,"encryption_key":encode_key,"checkpoint_choose_method":checkpoint_choose_method})

endpoint = f"{base_url}/model"

response = requests.post(endpoint,data=data,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())
assert "id" in response.json().keys()
model_id = response.json()["id"]

### List models <a class="anchor" id="head-5"></a>

In [ ]:
endpoint = f"{base_url}/model"

response = requests.get(endpoint, headers=headers)
assert response.status_code in (200, 201)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("model id\t\t\t     network architecture")
for rsp in response.json():
    rsp_keys = rsp.keys()
    assert "id" in rsp_keys and "network_arch" in rsp_keys
    print(rsp["id"],rsp["network_arch"])

### Assign train, eval datasets <a class="anchor" id="head-6"></a>

In [ ]:
dataset_information = {"train_datasets":[train_dataset_id],
                       "eval_dataset":eval_dataset_id,
                       "inference_dataset":test_dataset_id,
                       "calibration_dataset":train_dataset_id}

data = json.dumps(dataset_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())

### Assign PTM <a class="anchor" id="head-7"></a>

Search for the PTM on NGC for the Classification model chosen

In [ ]:
# List all pretrained models for the chosen network architecture
model_list = f"{base_url}/model"
response = requests.get(model_list, headers=headers)
assert response.status_code in (200, 201)

response_json = response.json()

for rsp in response_json:
    rsp_keys = rsp.keys()
    assert "network_arch" in rsp_keys
    if rsp["network_arch"] == model_name:
        if "encryption_key" not in rsp.keys():
            assert "name" in rsp_keys and "version" in rsp_keys and "ngc_path" in rsp_keys and "additional_id_info" in rsp_keys
            print(f'PTM Name: {rsp["name"]}; PTM version: {rsp["version"]}; NGC PATH: {rsp["ngc_path"]}; Additional info: {rsp["additional_id_info"]}')

In [ ]:
# Assigning pretrained models to different classification models
# From the output of previous cell make the appropriate changes to this map if you want to change the default PTM backbone.
# Changing the default backbone here requires changing default spec/config during train/eval etc like for example
# If you are changing the ptm to resnet34, then you have to modify the config key num_layers if it exists to 34 manually
pretrained_map = {"classification_tf1" : "pretrained_classification:resnet18",
                  "classification_tf2" : "pretrained_classification_tf2:efficientnet_b0",
                  "classification_pyt" : "pretrained_fan_classification_imagenet:fan_hybrid_tiny",
                  "multitask_classification" : "pretrained_classification:resnet10"}
no_ptm_models = set([])

In [ ]:
# Get pretrained model for classification
if model_name not in no_ptm_models:
    model_list = f"{base_url}/model"
    response = requests.get(model_list, headers=headers)
    assert response.status_code in (200, 201)

    response_json = response.json()

    # Search for ptm with given ngc path
    ptm = []
    for rsp in response_json:
        assert "network_arch" in rsp_keys and "ngc_path" in rsp_keys
        if rsp["network_arch"] == model_name and rsp["ngc_path"].endswith(pretrained_map[model_name]):
            assert "id" in rsp_keys
            ptm_id = rsp["id"]
            ptm = [ptm_id]
            print("Metadata for model with requested NGC Path")
            print(rsp)
            break

In [ ]:
if model_name not in no_ptm_models:
    ptm_information = {"ptm":ptm}
    data = json.dumps(ptm_information)

    endpoint = f"{base_url}/model/{model_id}"

    response = requests.patch(endpoint, data=data, headers=headers)
    assert response.status_code in (200, 201)

    print(response)
    print(response.json())

### View hyperparameters that are enabled for AutoML by default <a class="anchor" id="head-8"></a>

In [ ]:
if automl_enabled:
    # Get default spec schema
    endpoint = f"{base_url}/model/{model_id}/specs/train/schema"
    response = requests.get(endpoint, headers=headers)
    assert response.status_code in (200, 201)
    assert "automl_default_parameters" in response.json().keys()
    specs = response.json()["automl_default_parameters"]
    print(json.dumps(specs, sort_keys=True, indent=4))

### Actions <a class="anchor" id="head-10"></a>

For all actions:
1. Get default spec schema and derive the default values
2. Modify defaults if needed
3. Post spec dictionary to the service
4. Run model action
5. Monitor job using retrieve
6. Download results using job download endpoint (if needed)

### Train <a class="anchor" id="head-11"></a>

#### Set AutoML related configurations <a class="anchor" id="head-9"></a>
Refer to these hyper-links to see the parameters supported by each network and add more parameters if necessary in addition to the default automl enabled parameters:

[Classification TF1](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/classification_tf1/classification_tf1%20-%20train.csv), 
[Classification TF2](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/classification_tf2/classification_tf2%20-%20train.csv), 
[Classification Pytorch](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/classification_pyt/classification_pyt%20-%20train.csv), 
[Multitask classification](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/multitask_classification/multitask_classification%20-%20train.csv)

In [ ]:
if automl_enabled:
    # Choose any metric that is present in the kpi dictionary present in the model's status.json. 
    # Example status.json for each model can be found in the respective section in NVIDIA TAO DOCS here: https://docs.nvidia.com/tao/tao-toolkit/text/model_zoo/cv_models/index.html
    if model_name == "classification_pyt":
        metric = "loss"
    else:
        metric = "kpi" 

    additional_automl_parameters = [] #Refer to parameter list mentioned in the above links and add any extra parameter in addition to the default enabled ones
    remove_default_automl_parameters = [] #Remove any hyperparameters that are enabled by default for AutoML

    automl_information = {"automl_enabled":automl_enabled,
                          "automl_algorithm":automl_algorithm,
                          "metric":metric,
                          "automl_max_recommendations": 20, # Only for Bayesian
                          "automl_R": 27, # Only for Hyperband
                          "automl_nu": 3, # Only for Hyperband
                          "epoch_multiplier": 1, # Only for Hyperband
                          "automl_add_hyperparameters":str(additional_automl_parameters),
                          "automl_remove_hyperparameters":str(remove_default_automl_parameters)
                         }
    data = json.dumps(automl_information)

    endpoint = f"{base_url}/model/{model_id}"

    response = requests.patch(endpoint, data=data, headers=headers)
    assert response.status_code in (200, 201)


    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/train/schema"

response = requests.get(endpoint, headers=headers)
assert response.status_code in (200, 201)


print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Override any of the parameters listed in the previous cell as required
# Example for multitask-classification (for each network the parameter key might be different)
if model_name == "multitask_classification":
    specs["training_config"]["num_epochs"] = 10
    specs["gpus"] = 1
# Example for classification_pyt
elif model_name == "classification_pyt":
    specs["train"]["train_config"]["runner"]["max_epochs"] = 40
    specs["train"]["num_gpus"] = 1
    specs["gpus"] = 1
# Example for classification_tf1
elif model_name == "classification_tf1":
    specs["train_config"]["n_epochs"] = 80
    specs["gpus"] = 1
# Example for classification_tf2
elif model_name == "classification_tf2":
    specs["train"]["num_epochs"] = 80
    specs["gpus"] = 1

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/train"

response = requests.post(endpoint,data=data,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = None
actions = ["train"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)
assert response.status_code in (200, 201)
assert response.json()

print(response)
print(response.json())

job_map["train_" + model_name] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
# For automl: Training times for different models benchmarked on 1 GPU V100 machine can be found here: https://docs.nvidia.com/tao/tao-toolkit/text/automl/automl.html#results-of-automl-experiments

job_id = job_map["train_" + model_name]
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:    
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)

    if "error_desc" in response.json().keys() and response.json()["error_desc"] in ("Job not found", "No AutoML run found"):
        print("Job is being created")
        time.sleep(5)
        continue
    assert response.status_code in (200, 201)
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))
    assert "status" in response.json().keys() and response.json().get("status") != "Error"
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
## To Stop an AutoML JOB
#    1. Stop the 'Monitor job status by repeatedly running this cell' cell (the cell right before this cell) manually
#    2. Uncomment the snippet in the next cell and run the cell

In [ ]:
# if automl_enabled:
#     job_id = job_map["train_" + model_name]
#     endpoint = f"{base_url}/model/{model_id}/job/{job_id}/cancel"

#     response = requests.post(endpoint, headers=headers)
#     assert response.status_code in (200, 201)

#     print(response)
#     print(response.json())

In [ ]:
## Resume AutoML

In [ ]:
# Uncomment the below snippet if you want to resume an already stopped AutoML job and then run the 'Monitor job status by repeatedly running this cell' cell above (4th cell above from this cell)
# if automl_enabled:
#     job_id = job_map["train_" + model_name]
#     endpoint = f"{base_url}/model/{model_id}/job/{job_id}/resume"

#     response = requests.post(endpoint, headers=headers)
#     assert response.status_code in (200, 201)

#     print(response)
#     print(response.json())

### Download train job artifacts <a class="anchor" id="head-12"></a>

In [ ]:
# Example to list the files of the executed train job
job_id = job_map["train_" + model_name]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/list_files'

response = requests.get(endpoint, headers=headers)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# ## Patch the model with proper metric before training to run this cell; By default loss is used, but some models dont log the parameter under the name 'loss'

# # Download selective job contents once the above job shows "Done" status
# # Example to download selective files of train job (Note: will take time)
# endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download_selective_files'

# file_lists = [] # Choose file names from the previous cell where all the files for this job were listed
# best_model = False # Enable this to download the checkpoint of the best performing model w.r.t to the metric chosen before starting training
# latest_model = True # Enable this to download the latest checkpoint of the training job; Disable best_model to use latest_model

# params = {"file_lists": file_lists, "best_model": best_model, "latest_model": latest_model}

# # Save
# temptar = f'{job_id}.tar.gz'
# with requests.get(endpoint, headers=headers, params=params, stream=True) as r:
#     r.raise_for_status()
#     with open(temptar, 'wb') as f:
#         for chunk in r.iter_content(chunk_size=8192):
#             f.write(chunk)

# print("Untarring")
# # Untar to destination
# tar_command = f'tar -xvf {temptar} -C {workdir}/'
# os.system(tar_command)
# os.remove(temptar)
# print(f"Results at {workdir}/{job_id}")
# model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# Downloading train job takes a longer time, uncomment this cell if you want to still proceed
if download_jobs:
    !python3 -m pip install tqdm
    from tqdm import tqdm
    endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'
    temptar = f'{job_id}.tar.gz'

    # Perform a HEAD request to get the file size without downloading the content
    response = requests.head(endpoint, headers=headers)

    # Check if the request was successful and the 'Content-Length' header is present
    if response.status_code == 200 and 'Content-Length' in response.headers:
        expected_file_size = int(response.headers['Content-Length'])
        print(f"Excpected file size is {expected_file_size}")
    else:
        expected_file_size = None  # Set to None if the size couldn't be determined

    with tqdm(total=expected_file_size, unit='B', unit_scale=True) as progress_bar:
        while True:
            # Check if the file already exists
            headers_download_job = dict(headers)
            if os.path.exists(temptar):
                # Get the current file size
                file_size = os.path.getsize(temptar)
                print(f"File size of dowloaded content until now is {file_size}")

                # If the file size matches the expected size, break out of the loop
                if file_size >= (expected_file_size-1):
                    print("Download completed successfully.")
                    print("Untarring")
                    # Untar to destination
                    tar_command = f'tar -xf {temptar} -C {workdir}/'
                    os.system(tar_command)
                    os.remove(temptar)
                    print(f"Results at {workdir}/{job_id}")
                    model_downloaded_path = f"{workdir}/{job_id}"
                    break

                # Set the headers to resume the download from where it left off
                headers_download_job['Range'] = f'bytes={file_size}-'
            # Open the file for writing in binary mode
            with open(temptar, 'ab') as f:
                try:
                    response = requests.get(endpoint, headers=headers_download_job, stream=True)
                    print(response)
                    # Check if the request was successful
                    if response.status_code in [200, 206]:
                        # Iterate over the content in chunks
                        for chunk in response.iter_content(chunk_size=1024):
                            if chunk:
                                # Write the chunk to the file
                                f.write(chunk)
                                # Flush and sync the file to disk
                                f.flush()
                                os.fsync(f.fileno())
                            progress_bar.update(len(chunk))
                    else:
                        print(f"Failed to download file. Status code: {response.status_code}")
                except requests.exceptions.RequestException as e:
                    print("Connection interrupted during download, resuming download from breaking point")
                    time.sleep(5)  # Sleep for a while before retrying the request
                    continue  # Continue the loop to retry the request

In [ ]:
# View the checkpoints generated for the training job and for automl jobs, in addition view: best performing model's config and the results of all automl experiments

if download_jobs:
    if automl_enabled:
        !python3 -m pip install pandas==1.5.1
        import pandas as pd
        model_downloaded_path = f"{model_downloaded_path}/best_model"
        assert glob.glob(f"{model_downloaded_path}/*.protobuf") or glob.glob(f"{model_downloaded_path}/*.yaml")

    assert os.path.exists(model_downloaded_path)
    assert (glob.glob(model_downloaded_path + "/**/*.tlt", recursive=True) + glob.glob(model_downloaded_path + "/**/*.hdf5", recursive=True) + glob.glob(model_downloaded_path + "/**/*.pth", recursive=True))

    if os.path.exists(model_downloaded_path):        
        #List the binary model file
        print("\nCheckpoints for the training experiment")
        if os.path.exists(model_downloaded_path+"/train/weights") and len(os.listdir(model_downloaded_path+"/train/weights")) > 0:
            print(f"Folder: {model_downloaded_path}/train/weights")
            print("Files:", os.listdir(model_downloaded_path+"/train/weights"))
        elif os.path.exists(model_downloaded_path+"/weights") and len(os.listdir(model_downloaded_path+"/weights")) > 0:
            print(f"Folder: {model_downloaded_path}/weights")
            print("Files:", os.listdir(model_downloaded_path+"/weights"))
        else:
            print(f"Folder: {model_downloaded_path}")
            print("Files:", os.listdir(model_downloaded_path))

        if automl_enabled:
            assert glob.glob(f"{model_downloaded_path}/*.protobuf") or glob.glob(f"{model_downloaded_path}/*.yaml")
            experiment_artifacts = json.load(open(f"{model_downloaded_path}/controller.json","r"))
            data_frame = pd.DataFrame(experiment_artifacts)
            # Print experiment id/number and the corresponding result
            print("\nResults of all experiments")
            with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
                print(data_frame[["id","result"]])

### Evaluate <a class="anchor" id="head-12"></a>

In [ ]:
# Get model handler parameters
endpoint = f"{base_url}/model/{model_id}"
response = requests.get(endpoint, headers=headers)
assert response.status_code in (200, 201)
assert response.json()

model_parameters = response.json()
update_checkpoint_choosing = {}
update_checkpoint_choosing["checkpoint_choose_method"] = model_parameters["checkpoint_choose_method"]
update_checkpoint_choosing["checkpoint_epoch_number"] = model_parameters["checkpoint_epoch_number"]
print(update_checkpoint_choosing)

In [ ]:
# Change the method by which checkpoint from the parent action is chosen, when parent action is a train/retrain action.
# Example for evaluate action below, can be applied in the same way for other actions too
update_checkpoint_choosing["checkpoint_choose_method"] = "latest_model" # Choose between best_model/latest_model/from_epoch_number
# If from_epoch_number is chosen then assign the epoch number to the dictionary key in the format 'from_epoch_number{train_job_id}'
# update_checkpoint_choosing["checkpoint_epoch_number"]["from_epoch_number_28a2754e-50ef-43a8-9733-98913776dd90"] = 3
data = json.dumps(update_checkpoint_choosing)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)
assert response.status_code in (200, 201)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/evaluate/schema"

response = requests.get(endpoint, headers=headers)
assert response.status_code in (200, 201)

print(response)
#print(response.json()) ## Uncomment for verbose schema
assert "default" in response.json().keys()
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Modify specs dictionary to change any config parameters

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/evaluate"

response = requests.post(endpoint,data=data,headers=headers)
assert response.status_code in (200, 201)


print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train_" + model_name]
actions = ["evaluate"]
data = json.dumps({"job":parent,"actions":actions,"parent_id":model_id,"parent_job_type":"model"})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)
assert response.status_code in (200, 201)
assert response.json()

print(response)
print(response.json())

job_map["evaluate_" + model_name] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map["evaluate_" + model_name]
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:    
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    assert response.status_code in (200, 201)
    print(response)
    print(response.json())
    assert "status" in response.json().keys() and response.json().get("status") != "Error"
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### Prune, Retrain and Evaluation chaining <a class="anchor" id="head-13"></a>

- We optimize the trained model by pruning and retraining in the following cells

#### Apply specs for prune <a class="anchor" id="head-14"></a>

In [ ]:
if model_name != "classification_pyt":
    # Get default spec schema
    endpoint = f"{base_url}/model/{model_id}/specs/prune/schema"

    response = requests.get(endpoint, headers=headers)
    assert response.status_code in (200, 201)

    print(response)
    #print(response.json()) ## Uncomment for verbose schema
    assert "default" in response.json().keys()
    specs = response.json()["default"]
    print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
if model_name != "classification_pyt":
    # Apply changes to specs dictionary if required here
    if model_name == "classification_tf2":
        specs["prune"]["byom_model_path"] = ""

In [ ]:
if model_name != "classification_pyt":
    # Post spec
    data = json.dumps(specs)

    endpoint = f"{base_url}/model/{model_id}/specs/prune"

    response = requests.post(endpoint,data=data,headers=headers)
    assert response.status_code in (200, 201)

    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

#### Apply specs for retrain <a class="anchor" id="head-15"></a>

In [ ]:
if model_name != "classification_pyt":
    # Get default spec schema
    endpoint = f"{base_url}/model/{model_id}/specs/retrain/schema"

    response = requests.get(endpoint, headers=headers)
    assert response.status_code in (200, 201)

    print(response)
    #print(response.json()) ## Uncomment for verbose schema
    specs = response.json()["default"]
    print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
if model_name != "classification_pyt":
    # Override any of the parameters listed in the previous cell as required
    # Example for multitask-classification (for each network the parameter key might be different)
    if model_name == "multitask_classification":
        specs["training_config"]["num_epochs"] = 10
        specs["gpus"] = 1
    # Example for classification_tf1
    elif model_name == "classification_tf1":
        specs["train_config"]["n_epochs"] = 80
        specs["gpus"] = 1
    # Example for classification_tf2
    elif model_name == "classification_tf2":
        specs["train"]["num_epochs"] = 80
        specs["gpus"] = 1

In [ ]:
if model_name != "classification_pyt":
    # Post spec
    data = json.dumps(specs)

    endpoint = f"{base_url}/model/{model_id}/specs/retrain"

    response = requests.post(endpoint,data=data,headers=headers)
    assert response.status_code in (200, 201)

    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

#### Run Actions <a class="anchor" id="head-16"></a>

We use the API's job chaining feature to prune, retrain and evaluate the retrained model

In [ ]:
if model_name != "classification_pyt":
    # Run actions
    parent = job_map["train_" + model_name]
    actions = ["prune","retrain","evaluate"]
    data = json.dumps({"job":parent,"actions":actions,"parent_id":model_id,"parent_job_type":"model"})

    endpoint = f"{base_url}/model/{model_id}/job"

    response = requests.post(endpoint, data=data, headers=headers)
    assert response.status_code in (200, 201)
    assert response.json()

    print(response)
    print(response.json())

    job_map["prune_" + model_name] = response.json()[0]
    job_map["retrain_" + model_name] = response.json()[1]
    job_map["eval_retrain_" + model_name] = response.json()[2]
    print(job_map)

In [ ]:
if model_name != "classification_pyt":
    # Monitor job status by repeatedly running this cell (prune)
    job_id = job_map["prune_" + model_name]
    endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

    while True:    
        clear_output(wait=True)
        response = requests.get(endpoint, headers=headers)
        assert response.status_code in (200, 201)
        print(response)
        print(response.json())
        assert "status" in response.json().keys() and response.json().get("status") != "Error"
        if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
            break
        time.sleep(15)

In [ ]:
if model_name != "classification_pyt":
    # Monitor job status by repeatedly running this cell (retrain)
    job_id = job_map["retrain_" + model_name]
    endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

    while True:    
        clear_output(wait=True)
        response = requests.get(endpoint, headers=headers)
        assert response.status_code in (200, 201)
        print(response)
        print(response.json())
        assert "status" in response.json().keys() and response.json().get("status") != "Error"
        if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
            break
        time.sleep(15)

In [ ]:
if model_name != "classification_pyt":
    # Monitor job status by repeatedly running this cell (evaluate)
    job_id = job_map["eval_retrain_" + model_name]
    endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

    while True:    
        clear_output(wait=True)
        response = requests.get(endpoint, headers=headers)
        assert response.status_code in (200, 201)
        print(response)
        print(response.json())
        assert "status" in response.json().keys() and response.json().get("status") != "Error"
        if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
            break
        time.sleep(15)

In [ ]:
# if model_name != "classification_pyt":
#    # Optional cancel job - for jobs that are pending/running (retrain)

#     job_id = job_map["retrain_" + model_name]
#     endpoint = f"{base_url}/model/{model_id}/job/{job_id}/cancel"

#     response = requests.post(endpoint, headers=headers)
#     assert response.status_code in (200, 201)

#     print(response)
#     print(response.json())

In [ ]:
# if model_name != "classification_pyt":
    # # Optional delete job - for jobs that are error/done (retrain)

    # job_id = job_map["retrain_" + model_name]
    # endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

    # response = requests.delete(endpoint, headers=headers)
    # assert response.status_code in (200, 201)

    # print(response)
    # print(response.json())

### Export <a class="anchor" id="head-17"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/export/schema"

response = requests.get(endpoint, headers=headers)
assert response.status_code in (200, 201)

print(response)
#print(response.json()) ## Uncomment for verbose schema
assert "default" in response.json().keys()
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to spec dictionary if required

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/export"

response = requests.post(endpoint,data=data,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train_" + model_name]
actions = ["export"]
data = json.dumps({"job":parent,"actions":actions,"parent_id":model_id,"parent_job_type":"model"})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)
assert response.status_code in (200, 201)
assert response.json()

print(response)
print(response.json())

job_map["export_" + model_name] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map["export_" + model_name]
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    assert response.status_code in (200, 201)
    print(response)
    print(response.json())
    assert "status" in response.json().keys() and response.json().get("status") != "Error"
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### TRT Engine generation using TAO-Deploy <a class="anchor" id="head-19"></a>

- Here, we use the exported model to generate trt engine

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/gen_trt_engine/schema"

response = requests.get(endpoint, headers=headers)
assert response.status_code in (200, 201)

print(response)
#print(response.json()) ## Uncomment for verbose schema
assert "default" in response.json().keys()
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
if model_name == "classification_tf2":
    specs["gen_trt_engine"]["tensorrt"]["data_type"] = "int8"
elif model_name == "classification_pyt":
    specs["gen_trt_engine"]["tensorrt"]["data_type"] = "fp16"
else:
    specs["data_type"] = "int8"

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/gen_trt_engine"

response = requests.post(endpoint,data=data,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["export_" + model_name]
actions = ["gen_trt_engine"]
data = json.dumps({"job":parent,"actions":actions,"parent_id":model_id,"parent_job_type":"model"})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)
assert response.status_code in (200, 201)
assert response.json()

print(response)
print(response.json())

job_map["model_gen_trt_engine_" + model_name] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map["model_gen_trt_engine_" + model_name]
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    assert response.status_code in (200, 201)
    print(response)
    print(response.json())
    assert "status" in response.json().keys() and response.json().get("status") != "Error"
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### TAO inference <a class="anchor" id="head-20"></a>

- Run inference on a set of images using the .tlt model created at train step

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/inference/schema"

response = requests.get(endpoint, headers=headers)
assert response.status_code in (200, 201)

print(response)
#print(response.json()) ## Uncomment for verbose schema
assert "default" in response.json().keys()
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to the specs you want to modify

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/inference"

response = requests.post(endpoint,data=data,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train_" + model_name]
actions = ["inference"]
data = json.dumps({"job":parent,"actions":actions,"parent_id":model_id,"parent_job_type":"model"})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)
assert response.status_code in (200, 201)
assert response.json()

print(response)
print(response.json())

job_map["inference_tlt_" + model_name] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map["inference_tlt_" + model_name]
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    assert response.status_code in (200, 201)
    print(response)
    print(response.json())
    assert "status" in response.json().keys() and response.json().get("status") != "Error"
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
if download_jobs:
    job_id = job_map["inference_tlt_" + model_name]
    !python3 -m pip install tqdm
    from tqdm import tqdm
    endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'
    temptar = f'{job_id}.tar.gz'

    # Perform a HEAD request to get the file size without downloading the content
    response = requests.head(endpoint, headers=headers)

    # Check if the request was successful and the 'Content-Length' header is present
    if response.status_code == 200 and 'Content-Length' in response.headers:
        expected_file_size = int(response.headers['Content-Length'])
        print(f"Excpected file size is {expected_file_size}")
    else:
        expected_file_size = None  # Set to None if the size couldn't be determined

    with tqdm(total=expected_file_size, unit='B', unit_scale=True) as progress_bar:
        while True:
            # Check if the file already exists
            headers_download_job = dict(headers)
            if os.path.exists(temptar):
                # Get the current file size
                file_size = os.path.getsize(temptar)
                print(f"File size of dowloaded content until now is {file_size}")

                # If the file size matches the expected size, break out of the loop
                if file_size >= (expected_file_size-1):
                    print("Download completed successfully.")
                    print("Untarring")
                    # Untar to destination
                    tar_command = f'tar -xf {temptar} -C {workdir}/'
                    os.system(tar_command)
                    os.remove(temptar)
                    print(f"Results at {workdir}/{job_id}")
                    inference_out_path = f"{workdir}/{job_id}"
                    break

                # Set the headers to resume the download from where it left off
                headers_download_job['Range'] = f'bytes={file_size}-'
            # Open the file for writing in binary mode
            with open(temptar, 'ab') as f:
                try:
                    response = requests.get(endpoint, headers=headers_download_job, stream=True)
                    print(response)
                    # Check if the request was successful
                    if response.status_code in [200, 206]:
                        # Iterate over the content in chunks
                        for chunk in response.iter_content(chunk_size=1024):
                            if chunk:
                                # Write the chunk to the file
                                f.write(chunk)
                                # Flush and sync the file to disk
                                f.flush()
                                os.fsync(f.fileno())
                            progress_bar.update(len(chunk))
                    else:
                        print(f"Failed to download file. Status code: {response.status_code}")
                except requests.exceptions.RequestException as e:
                    print("Connection interrupted during download, resuming download from breaking point")
                    time.sleep(5)  # Sleep for a while before retrying the request
                    continue  # Continue the loop to retry the request

In [ ]:
# Print Classification results
if download_jobs:
    if model_name == "classification_tf1":
        assert os.path.exists(f'{inference_out_path}/result.csv')
        !cat {inference_out_path}/result.csv
    elif "classification_" in model_name:
        assert os.path.exists(f'{inference_out_path}/inference/result.csv')
        !cat {inference_out_path}/inference/result.csv
    elif model_name == "multitask_classification":
        assert os.path.exists(f'{inference_out_path}/result.txt')
        !cat {inference_out_path}/result.txt

### TRT inference <a class="anchor" id="head-21"></a>

- no need to change the specs since we already uploaded it at the tlt inference step

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/inference/schema"

response = requests.get(endpoint, headers=headers)
assert response.status_code in (200, 201)

print(response)
#print(response.json()) ## Uncomment for verbose schema
assert "default" in response.json().keys()
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to the specs you want to modify

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/inference"

response = requests.post(endpoint,data=data,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["model_gen_trt_engine_" + model_name]
actions = ["inference"]
data = json.dumps({"job":parent,"actions":actions,"parent_id":model_id,"parent_job_type":"model"})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)
assert response.status_code in (200, 201)
assert response.json()

print(response)
print(response.json())

job_map["inference_trt_" + model_name] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map["inference_trt_" + model_name]
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    assert response.status_code in (200, 201)
    print(response)
    print(response.json())
    assert "status" in response.json().keys() and response.json().get("status") != "Error"
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
if download_jobs:
    job_id = job_map["inference_trt_" + model_name]
    !python3 -m pip install tqdm
    from tqdm import tqdm
    endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'
    temptar = f'{job_id}.tar.gz'

    # Perform a HEAD request to get the file size without downloading the content
    response = requests.head(endpoint, headers=headers)

    # Check if the request was successful and the 'Content-Length' header is present
    if response.status_code == 200 and 'Content-Length' in response.headers:
        expected_file_size = int(response.headers['Content-Length'])
        print(f"Excpected file size is {expected_file_size}")
    else:
        expected_file_size = None  # Set to None if the size couldn't be determined

    with tqdm(total=expected_file_size, unit='B', unit_scale=True) as progress_bar:
        while True:
            # Check if the file already exists
            headers_download_job = dict(headers)
            if os.path.exists(temptar):
                # Get the current file size
                file_size = os.path.getsize(temptar)
                print(f"File size of dowloaded content until now is {file_size}")

                # If the file size matches the expected size, break out of the loop
                if file_size >= (expected_file_size-1):
                    print("Download completed successfully.")
                    print("Untarring")
                    # Untar to destination
                    tar_command = f'tar -xf {temptar} -C {workdir}/'
                    os.system(tar_command)
                    os.remove(temptar)
                    print(f"Results at {workdir}/{job_id}")
                    inference_out_path = f"{workdir}/{job_id}"
                    break

                # Set the headers to resume the download from where it left off
                headers_download_job['Range'] = f'bytes={file_size}-'
            # Open the file for writing in binary mode
            with open(temptar, 'ab') as f:
                try:
                    response = requests.get(endpoint, headers=headers_download_job, stream=True)
                    print(response)
                    # Check if the request was successful
                    if response.status_code in [200, 206]:
                        # Iterate over the content in chunks
                        for chunk in response.iter_content(chunk_size=1024):
                            if chunk:
                                # Write the chunk to the file
                                f.write(chunk)
                                # Flush and sync the file to disk
                                f.flush()
                                os.fsync(f.fileno())
                            progress_bar.update(len(chunk))
                    else:
                        print(f"Failed to download file. Status code: {response.status_code}")
                except requests.exceptions.RequestException as e:
                    print("Connection interrupted during download, resuming download from breaking point")
                    time.sleep(5)  # Sleep for a while before retrying the request
                    continue  # Continue the loop to retry the request

In [ ]:
# Print Classification results
if download_jobs:
    if model_name in ("classification_tf1", "multitask_classification"):
        !cat {inference_out_path}/result.csv
    elif "classification_" in model_name:
        !cat {inference_out_path}/inference/result.csv

### Delete model <a class="anchor" id="head-21"></a>

In [ ]:
endpoint = f"{base_url}/model/{model_id}"

response = requests.delete(endpoint,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())

### Delete dataset <a class="anchor" id="head-21"></a>

#### Delete train dataset <a class="anchor" id="head-21"></a>

In [ ]:
endpoint = f"{base_url}/dataset/{train_dataset_id}"

response = requests.delete(endpoint,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())

#### Delete val dataset <a class="anchor" id="head-21"></a>

In [ ]:
endpoint = f"{base_url}/dataset/{eval_dataset_id}"

response = requests.delete(endpoint,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())

#### Delete test dataset <a class="anchor" id="head-21"></a>

In [ ]:
endpoint = f"{base_url}/dataset/{test_dataset_id}"

response = requests.delete(endpoint,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())